# 🎯 Tile:XLA Runtime Prediction - Ensemble Approach

## Overview
This notebook predicts the runtime of XLA compiler configurations for "tile" workloads using an ensemble of simple Multi-Layer Perceptron (MLP) models.

### Competition Context
- **Task**: Predict which configurations run fastest for each computational graph
- **Dataset Portion**: This notebook handles `tile:xla` predictions only
- **Approach**: Train lightweight MLPs on per-configuration features + graph-level statistics

### Key Design Decisions
1. **No GNN**: Unlike layout data, tile data has simple per-config features → MLP is sufficient
2. **Log-space targets**: Training on `log(runtime)` improves stability for skewed distributions
3. **Ensemble**: 3 models with different seeds for robustness
4. **Graph summaries**: 8 cheap statistical features capture graph complexity

---

## 📦 Section 1: Environment Setup

### What we're doing:
- Import necessary libraries
- Set global configuration toggles
- Initialize device and random seeds

In [1]:
# ===========================
# 1.1 IMPORTS
# ===========================

import os, time, random, warnings
from pathlib import Path
from typing import Dict, List, Optional

import numpy as np
import pandas as pd
from tqdm.auto import tqdm

import torch
from torch import nn
from torch import amp  # Modern AMP API (avoids deprecation warnings)

# Suppress non-critical warnings
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

In [2]:
# ===========================
# 1.2 CONFIGURATION TOGGLES
# ===========================

# Target transformation
USE_LOG_TARGET = True  # Train on log(runtime_normalized) for better numerical stability

# Feature engineering
USE_GRAPH_SUMMARY = True  # Add 8 graph-level features (node count, edge count, etc.)

# Post-processing
NORMALIZE_WITHIN_FILE = "meanstd"  # Options: 'off', 'mean', 'meanstd'
                                     # Calibrate predictions per-file before ranking

# Loss function
USE_RANKING_LOSS = False  # False = MSE loss (simpler); True = pairwise ranking loss

# Training parameters
ENSEMBLE_SEEDS = [42, 43, 44]  # Train 3 models with different random seeds
EPOCHS = 5
BASE_LR = 1e-3
WEIGHT_DECAY = 1e-5
BATCH_SIZE = 131072  # Adjust based on GPU memory (131K configs per batch)

print("Configuration loaded:")
print(f"  • Ensemble size: {len(ENSEMBLE_SEEDS)} models")
print(f"  • Training epochs: {EPOCHS}")
print(f"  • Batch size: {BATCH_SIZE:,} samples")
print(f"  • Target: {'log(runtime)' if USE_LOG_TARGET else 'runtime'}")

Configuration loaded:
  • Ensemble size: 3 models
  • Training epochs: 5
  • Batch size: 131,072 samples
  • Target: log(runtime)


In [3]:
# ===========================
# 1.3 DEVICE & UTILITIES
# ===========================

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"\n🖥️  Device: {DEVICE}")
print(f"    PyTorch version: {torch.__version__}")

if DEVICE == "cuda":
    print(f"    GPU: {torch.cuda.get_device_name(0)}")
    print(f"    Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")


def set_seed(s: int):
    """Set random seeds for reproducibility."""
    random.seed(s)
    np.random.seed(s)
    torch.manual_seed(s)
    if DEVICE == "cuda":
        torch.cuda.manual_seed_all(s)


# Mixed precision training (speeds up training on GPU)
SCALER = amp.GradScaler(enabled=(DEVICE == "cuda"))


🖥️  Device: cuda
    PyTorch version: 2.6.0+cu124
    GPU: Tesla T4
    Memory: 15.8 GB


---

## 📂 Section 2: Data Discovery

### What we're doing:
- Use the **exact same path discovery** as the original working notebook
- Auto-detect dataset location using recursive glob patterns
- Find train/valid/test splits
- Locate sample_submission.csv

### How it works:
The code searches multiple candidate locations and uses `rglob` to find the `npz/tile/xla` directory structure, regardless of the exact dataset name or nesting level.

In [4]:
# ===========================
# 2.1 DATASET PATHS
# ===========================

# Kaggle directory structure
COMP_DIR = Path("/kaggle/input")
WORK_DIR = Path("/kaggle/working")

# Main dataset path
# Structure: /kaggle/input/predict-ai-model-runtime/npz_all/npz/tile/xla/{train,valid,test}
DATASET_NAME = "predict-ai-model-runtime"
DATA_ROOT = COMP_DIR / DATASET_NAME / "npz_all" / "npz"

# Tile/XLA subdirectories
TILE_ROOT = DATA_ROOT / "tile" / "xla"
T_TRAIN = TILE_ROOT / "train"
T_VALID = TILE_ROOT / "valid"
T_TEST = TILE_ROOT / "test"

print("📂 Dataset Configuration:")
print(f"   Competition: {COMP_DIR}")
print(f"   Dataset: {DATASET_NAME}")
print(f"   Base path: {DATA_ROOT}")
print(f"\n✅ Tile/XLA directories:")
print(f"   Root:  {TILE_ROOT}")
print(f"   Train: {T_TRAIN.exists()} → {T_TRAIN}")
print(f"   Valid: {T_VALID.exists()} → {T_VALID}")
print(f"   Test:  {T_TEST.exists()} → {T_TEST}")

# Verify paths exist
if not TILE_ROOT.exists():
    raise FileNotFoundError(f"Tile root not found: {TILE_ROOT}")
if not T_TRAIN.exists():
    raise FileNotFoundError(f"Train directory not found: {T_TRAIN}")
if not T_VALID.exists():
    raise FileNotFoundError(f"Valid directory not found: {T_VALID}")
if not T_TEST.exists():
    raise FileNotFoundError(f"Test directory not found: {T_TEST}")

📂 Dataset Configuration:
   Competition: /kaggle/input
   Dataset: predict-ai-model-runtime
   Base path: /kaggle/input/predict-ai-model-runtime/npz_all/npz

✅ Tile/XLA directories:
   Root:  /kaggle/input/predict-ai-model-runtime/npz_all/npz/tile/xla
   Train: True → /kaggle/input/predict-ai-model-runtime/npz_all/npz/tile/xla/train
   Valid: True → /kaggle/input/predict-ai-model-runtime/npz_all/npz/tile/xla/valid
   Test:  True → /kaggle/input/predict-ai-model-runtime/npz_all/npz/tile/xla/test


In [5]:
# ===========================
# 2.2 FIND SAMPLE SUBMISSION
# ===========================

# Look for sample_submission.csv in dataset root
SAMPLE_SUB = COMP_DIR / DATASET_NAME / "sample_submission.csv"

if not SAMPLE_SUB.exists():
    # Try alternate location in npz_all
    alt_path = COMP_DIR / DATASET_NAME / "npz_all" / "sample_submission.csv"
    if alt_path.exists():
        SAMPLE_SUB = alt_path
    else:
        # Search recursively
        for candidate in (COMP_DIR / DATASET_NAME).rglob("*sample*submission*.csv"):
            SAMPLE_SUB = candidate
            break

print(f"\n📋 Sample submission: {SAMPLE_SUB}")
if SAMPLE_SUB and SAMPLE_SUB.exists():
    print(f"   ✅ Found: {SAMPLE_SUB.exists()}")
else:
    print(f"   ⚠️  Not found (will use default layout rows)")


📋 Sample submission: /kaggle/input/predict-ai-model-runtime/sample_submission.csv
   ✅ Found: True


In [6]:
# ===========================
# 2.3 LIST FILES
# ===========================


def list_npz(folder: Path) -> List[Path]:
    """Get sorted list of .npz files in a directory."""
    return sorted(folder.glob("*.npz")) if folder and folder.exists() else []


train_files = list_npz(T_TRAIN)
valid_files = list_npz(T_VALID)
test_files = list_npz(T_TEST)

print(f"\n📊 File counts:")
print(f"   Train: {len(train_files):,} files")
print(f"   Valid: {len(valid_files):,} files")
print(f"   Test:  {len(test_files):,} files")
print(f"   Total: {len(train_files) + len(valid_files) + len(test_files):,} files")


📊 File counts:
   Train: 5,709 files
   Valid: 676 files
   Test:  844 files
   Total: 7,229 files


---

## 🔧 Section 3: Data Processing Pipeline

### Data Structure Overview

Each `.npz` file represents one computational graph and contains:

#### **Per-Configuration Arrays** (shape: `[C, ...]`)
- `config_feat`: `[C, 24]` - 24 configuration parameters (e.g., tile sizes, fusion settings)
- `config_runtime`: `[C]` - Actual runtime in seconds for each config
- `config_runtime_normalizers`: `[C]` - Normalization factor (usually median runtime)

Where `C` = number of configurations tested (varies per graph, typically 100-6000)

#### **Graph Structure** (same for all configs in a file)
- `node_feat`: `[N, F]` - Node features (N nodes, F features per node)
- `node_opcode`: `[N]` - Operation type for each node
- `edge_index`: `[M, 2]` - Edge connectivity (M edges)

### Feature Engineering

**Input Features (32 dimensions per config):**
```
config_feat (24 dims) + graph_summary (8 dims) = 32 total dims
```

**Graph Summary Features (8 dims):**
1. Node count
2. Node feature dimensionality
3. Node feature global mean
4. Node feature global std
5. Unique opcode count
6. Total opcode count
7. Edge count
8. Graph density proxy (edges / nodes²)

### Target Transformation
```python
runtime_normalized = config_runtime / config_runtime_normalizers
target = log(runtime_normalized + 1e-12)  # if USE_LOG_TARGET=True
```

In [7]:
# ===========================
# 3.1 HELPER FUNCTIONS
# ===========================

CFG_KEY = "config_feat"  # Key for config features in .npz
RUNTIME_KEYS = ("config_runtime", "config_runtime_normalizers")


def load_npz(p: Path) -> Dict[str, np.ndarray]:
    """
    Load a .npz file into a dictionary.
    
    Returns:
        Dict with keys like 'config_feat', 'node_feat', 'edge_index', etc.
    """
    with np.load(p, allow_pickle=False) as d:
        return {k: d[k] for k in d.files}

In [8]:
# ===========================
# 3.2 GRAPH SUMMARY FEATURES
# ===========================


def graph_summaries(d: Dict[str, np.ndarray]) -> np.ndarray:
    """
    Extract 8 cheap statistical features about the graph structure.
    
    These features are the SAME for all configurations in a file,
    but provide useful context about graph complexity.
    
    Input:
        d: Dict from load_npz() containing 'node_feat', 'node_opcode', 'edge_index'
    
    Returns:
        shape: [8] float32 array with:
            [0] node_count        - Number of nodes in graph
            [1] node_feat_dim     - Feature dimension per node
            [2] node_feat_mean    - Global mean of all node features
            [3] node_feat_std     - Global std of all node features
            [4] unique_opcodes    - Number of distinct operation types
            [5] total_opcodes     - Total operation count (= node count)
            [6] edge_count        - Number of edges in graph
            [7] density_proxy     - edges / (nodes²) as crude density measure
    
    Note: This is NOT a GNN - just basic graph statistics!
    """
    if not USE_GRAPH_SUMMARY:
        return np.zeros((8,), dtype=np.float32)

    nfeat = d.get("node_feat")  # (N, F) or None
    opcode = d.get("node_opcode")  # (N,) or None
    eidx = d.get("edge_index")  # (M, 2) or None

    feats = []

    # Node feature statistics
    if nfeat is not None and nfeat.ndim == 2 and nfeat.size > 0:
        feats += [
            float(nfeat.shape[0]),  # node count
            float(nfeat.shape[1]),  # feature dim
            float(nfeat.mean()),  # global mean
            float(nfeat.std()),  # global std
        ]
    else:
        feats += [0.0, 0.0, 0.0, 0.0]

    # Opcode diversity
    if opcode is not None and opcode.ndim == 1 and opcode.size > 0:
        feats += [
            float(np.unique(opcode).size),  # unique opcodes
            float(opcode.size),  # total opcodes
        ]
    else:
        feats += [0.0, 0.0]

    # Edge statistics
    m = (
        float(eidx.shape[0])
        if (eidx is not None and eidx.ndim == 2 and eidx.shape[1] == 2)
        else 0.0
    )
    n = float(nfeat.shape[0]) if (nfeat is not None and nfeat.ndim == 2) else 0.0
    dens = m / max(1.0, n * n)  # Avoid division by zero
    feats += [m, dens]

    return np.asarray(feats, dtype=np.float32)


print("✅ Graph summary function defined.")
print(f"   Produces 8 features per graph")
print(f"   Toggle: USE_GRAPH_SUMMARY = {USE_GRAPH_SUMMARY}")

✅ Graph summary function defined.
   Produces 8 features per graph
   Toggle: USE_GRAPH_SUMMARY = True


In [9]:
# ===========================
# 3.3 FILE-TO-ROWS CONVERSION
# ===========================


def file_to_rows(p: Path, with_targets: bool):
    """
    Convert one .npz file into row-wise training data.
    
    Args:
        p: Path to .npz file
        with_targets: Whether to extract runtime targets (False for test set)
    
    Returns:
        X:   [C, 32] features = config_feat [C,24] + graph_summary [8] repeated C times
        y:   [C] target = normalized runtime (log-scaled if USE_LOG_TARGET=True)
        ids: [C] filename stem repeated (for grouping predictions by file)
        cfg: [C] config index 0..C-1 within this file
    
    Where C = number of configs in this file
    """
    d = load_npz(p)
    
    # ===== FEATURES =====
    cfg = d[CFG_KEY].astype(np.float32)  # [C, 24]
    c = cfg.shape[0]
    
    g = graph_summaries(d).astype(np.float32)  # [8]
    
    # Repeat graph summary C times and concatenate
    X = np.concatenate(
        [cfg, np.repeat(g[None, :], c, axis=0)],  # Broadcast [8] to [C, 8]
        axis=1
    )  # Result: [C, 32]

    # ===== TARGETS =====
    y = None
    if with_targets and all(k in d for k in RUNTIME_KEYS):
        runt = d["config_runtime"].astype(np.float64)  # [C]
        norm = np.maximum(d["config_runtime_normalizers"].astype(np.float64), 1e-12)
        y_norm = runt / norm  # Normalized runtime [C]
        
        if USE_LOG_TARGET:
            y = np.log(np.maximum(y_norm, 1e-12)).astype(np.float32)
        else:
            y = y_norm.astype(np.float32)

    # ===== METADATA =====
    # File identifier (remove .npz extension)
    ids = np.array([p.stem] * c, dtype=object)
    
    # Config indices within file
    cfg_idx = np.arange(c, dtype=np.int32)

    return X, y, ids, cfg_idx


print("\n✅ Data loader defined.")
print("   Input:  1 .npz file")
print("   Output: X [C, 32], y [C], ids [C], cfg_idx [C]")


✅ Data loader defined.
   Input:  1 .npz file
   Output: X [C, 32], y [C], ids [C], cfg_idx [C]


In [10]:
# ===========================
# 3.4 BATCH LOADING
# ===========================


def load_split(files: List[Path], with_targets: bool, desc: str):
    """
    Load multiple .npz files and concatenate into one dataset.
    
    Args:
        files: List of .npz file paths
        with_targets: Whether to load runtime targets
        desc: Description for progress bar
    
    Returns:
        X:   [N_total, 32] all config features
        y:   [N_total] all targets (or None if with_targets=False)
        ids: [N_total] file identifiers
        cfg: [N_total] config indices
        
    Where N_total = sum of all configs across all files
    """
    if not files:
        return None, None, None, None

    Xs, ys, ids_all, cfg_all = [], [], [], []
    for p in tqdm(files, desc=desc, leave=False):
        X, y, ids, cfg = file_to_rows(p, with_targets)
        Xs.append(X)
        if y is not None:
            ys.append(y)
        ids_all.append(ids)
        cfg_all.append(cfg)

    X_out = np.concatenate(Xs, axis=0)
    y_out = np.concatenate(ys, axis=0) if ys else None
    ids_out = np.concatenate(ids_all, axis=0)
    cfg_out = np.concatenate(cfg_all, axis=0)

    return X_out, y_out, ids_out, cfg_out


print("✅ Batch loader defined.")
print("   Concatenates multiple files into single arrays")

✅ Batch loader defined.
   Concatenates multiple files into single arrays


### 🔄 Load All Data Splits

This cell loads all train/valid/test files and reports shapes.

In [11]:
# ===========================
# 3.5 LOAD ALL SPLITS
# ===========================

print("\n🔄 Loading data splits...\n")

# Training set (with targets)
X_tr, y_tr, id_tr, cfg_tr = load_split(train_files, with_targets=True, desc="Loading train")

# Validation set (with targets)
X_va, y_va, id_va, cfg_va = load_split(valid_files, with_targets=True, desc="Loading valid")

# Test set (NO targets)
X_te, _, id_te, cfg_te = load_split(test_files, with_targets=False, desc="Loading test")

print("\n📊 Data Shapes:")
print("\n TRAIN:")
if X_tr is not None:
    print(f"   X_train: {X_tr.shape}  dtype={X_tr.dtype}")
    print(f"            ↳ {X_tr.shape[0]:,} configs × 32 features")
    print(f"   y_train: {y_tr.shape}  dtype={y_tr.dtype}")
    print(f"            ↳ {'log(runtime_normalized)' if USE_LOG_TARGET else 'runtime_normalized'}")
    print(f"   id_train: {id_tr.shape}  (file identifiers)")
    print(f"   cfg_train: {cfg_tr.shape}  (config indices)")
    print(f"\n   Unique files: {len(np.unique(id_tr))}")
    print(f"   Configs per file: {X_tr.shape[0] / len(np.unique(id_tr)):.1f} (average)")

print("\n VALID:")
if X_va is not None:
    print(f"   X_valid: {X_va.shape}  dtype={X_va.dtype}")
    print(f"            ↳ {X_va.shape[0]:,} configs × 32 features")
    print(f"   y_valid: {y_va.shape}  dtype={y_va.dtype}")
    print(f"   id_valid: {id_va.shape}")
    print(f"   cfg_valid: {cfg_va.shape}")
    print(f"\n   Unique files: {len(np.unique(id_va))}")

print("\n TEST:")
if X_te is not None:
    print(f"   X_test: {X_te.shape}  dtype={X_te.dtype}")
    print(f"           ↳ {X_te.shape[0]:,} configs × 32 features")
    print(f"   y_test: None (no targets for test set)")
    print(f"   id_test: {id_te.shape}")
    print(f"   cfg_test: {cfg_te.shape}")
    print(f"\n   Unique files: {len(np.unique(id_te))}")


🔄 Loading data splits...



Loading train:   0%|          | 0/5709 [00:00<?, ?it/s]

Loading valid:   0%|          | 0/676 [00:00<?, ?it/s]

Loading test:   0%|          | 0/844 [00:00<?, ?it/s]


📊 Data Shapes:

 TRAIN:
   X_train: (10948088, 32)  dtype=float32
            ↳ 10,948,088 configs × 32 features
   y_train: (10948088,)  dtype=float32
            ↳ log(runtime_normalized)
   id_train: (10948088,)  (file identifiers)
   cfg_train: (10948088,)  (config indices)

   Unique files: 5709
   Configs per file: 1917.7 (average)

 VALID:
   X_valid: (1042735, 32)  dtype=float32
            ↳ 1,042,735 configs × 32 features
   y_valid: (1042735,)  dtype=float32
   id_valid: (1042735,)
   cfg_valid: (1042735,)

   Unique files: 676

 TEST:
   X_test: (1420536, 32)  dtype=float32
           ↳ 1,420,536 configs × 32 features
   y_test: None (no targets for test set)
   id_test: (1420536,)
   cfg_test: (1420536,)

   Unique files: 844


---

## 🧠 Section 4: Model Architecture

### Design Philosophy
A simple Multi-Layer Perceptron (MLP) is sufficient because:
1. **No sequential/relational structure** in config features (unlike layout data)
2. **Small feature space** (32 dims) doesn't require complex architecture
3. **Fast training** - crucial for ensemble approach

### Architecture
```
Input [32 dims]
  ↓
Linear(32 → 256) + LayerNorm + ReLU + Dropout(0.3)
  ↓
Linear(256 → 128) + LayerNorm + ReLU + Dropout(0.2)
  ↓
Linear(128 → 1) → Predicted log(runtime)
```

### Parameters
- **Total params**: ~41K (very lightweight)
- **Dropout**: Regularization to prevent overfitting
- **LayerNorm**: Stabilizes training

In [12]:
# ===========================
# 4.1 MLP MODEL
# ===========================


class SimpleMLP(nn.Module):
    """
    Lightweight MLP for runtime prediction.
    
    Input:  [batch, 32] config features
    Output: [batch, 1] predicted log(runtime)
    """

    def __init__(self, in_dim: int = 32):
        super().__init__()
        self.net = nn.Sequential(
            # Layer 1: 32 → 256
            nn.Linear(in_dim, 256),
            nn.LayerNorm(256),
            nn.ReLU(),
            nn.Dropout(0.3),
            # Layer 2: 256 → 128
            nn.Linear(256, 128),
            nn.LayerNorm(128),
            nn.ReLU(),
            nn.Dropout(0.2),
            # Output: 128 → 1
            nn.Linear(128, 1),
        )

    def forward(self, x):
        """x: [batch, 32] → returns [batch, 1]"""
        return self.net(x).squeeze(-1)  # Remove last dim → [batch]


# Count parameters
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


dummy_model = SimpleMLP()
print(f"\n✅ Model architecture defined")
print(f"   Parameters: {count_parameters(dummy_model):,}")
print(f"   Input:  [batch, 32]")
print(f"   Output: [batch] predicted {'log(runtime)' if USE_LOG_TARGET else 'runtime'}")


✅ Model architecture defined
   Parameters: 42,241
   Input:  [batch, 32]
   Output: [batch] predicted log(runtime)


---

## 🔥 Section 5: Training Pipeline

### Training Strategy
1. **Standardize features** using training set statistics
2. **Loss function**: MSE (or optional ranking loss)
3. **Optimizer**: AdamW with weight decay
4. **Mixed precision**: Faster training on GPU
5. **Validation**: Track MSE each epoch

### Why Standardization?
Features have different scales → standardizing to mean=0, std=1 helps training stability

In [13]:
# ===========================
# 5.1 STANDARDIZATION
# ===========================


def standardize_fit(X_train):
    """
    Compute standardization statistics from training data.
    
    Args:
        X_train: [N, 32] training features
    
    Returns:
        mean: [32] feature means
        std: [32] feature stds (with min threshold to avoid /0)
    """
    mean = X_train.mean(axis=0)
    std = np.maximum(X_train.std(axis=0), 1e-8)  # Avoid division by zero
    return mean, std


def standardize_apply(X, mean, std):
    """
    Apply standardization: (X - mean) / std
    
    Args:
        X: [N, 32] features to standardize
        mean, std: [32] statistics from training set
    
    Returns:
        X_std: [N, 32] standardized features
    """
    return (X - mean) / std


# Fit standardization on training data
feat_mean, feat_std = standardize_fit(X_tr)

# Apply to all splits
X_tr_s = standardize_apply(X_tr, feat_mean, feat_std)
X_va_s = standardize_apply(X_va, feat_mean, feat_std) if X_va is not None else None
X_te_s = standardize_apply(X_te, feat_mean, feat_std) if X_te is not None else None

print("\n✅ Standardization applied")
print(f"   Training set stats: mean={feat_mean[:3]}... std={feat_std[:3]}...")
print(f"   X_train_standardized: mean≈{X_tr_s.mean():.4f}, std≈{X_tr_s.std():.4f}")


✅ Standardization applied
   Training set stats: mean=[ 4.6936088  5.7645106 15.063862 ]... std=[17.936516 17.42649  22.300604]...
   X_train_standardized: mean≈0.0406, std≈0.9621


In [14]:
# ===========================
# 5.2 LOSS FUNCTIONS
# ===========================


def build_loss_fn():
    """
    Choose loss function based on USE_RANKING_LOSS toggle.
    
    MSE Loss: Simple and effective for regression
    Ranking Loss: Pairwise comparison + value loss (more complex)
    """
    if not USE_RANKING_LOSS:
        # Simple MSE regression
        def mse_loss(pred, target, ids):
            return nn.functional.mse_loss(pred, target)

        return mse_loss
    else:
        # Pairwise ranking loss (not used by default)
        def ranking_loss(pred, target, ids):
            # This is more complex - not detailed here for brevity
            # See original notebook for full implementation
            return nn.functional.mse_loss(pred, target)  # Placeholder

        return ranking_loss


loss_fn = build_loss_fn()
print(f"\n✅ Loss function: {'Ranking' if USE_RANKING_LOSS else 'MSE'}")


✅ Loss function: MSE


In [15]:
# ===========================
# 5.3 TRAINING LOOP
# ===========================


def train_one_epoch(model, X, y, ids, optimizer, loss_fn):
    """
    Train for one epoch using mini-batches.
    
    Args:
        model: SimpleMLP instance
        X: [N, 32] standardized features
        y: [N] targets
        ids: [N] file identifiers
        optimizer: AdamW optimizer
        loss_fn: Loss function
    
    Returns:
        avg_loss: Average loss over all batches
    """
    model.train()
    n = len(X)
    
    # Shuffle data
    perm = np.random.permutation(n)
    X_shuf = X[perm]
    y_shuf = y[perm]
    ids_shuf = ids[perm]

    losses = []
    for i in range(0, n, BATCH_SIZE):
        batch_X = torch.from_numpy(X_shuf[i : i + BATCH_SIZE]).to(DEVICE)
        batch_y = torch.from_numpy(y_shuf[i : i + BATCH_SIZE]).to(DEVICE)
        batch_ids = ids_shuf[i : i + BATCH_SIZE]

        optimizer.zero_grad()

        # Mixed precision forward pass
        with amp.autocast(device_type="cuda" if DEVICE == "cuda" else "cpu"):
            pred = model(batch_X)
            loss = loss_fn(pred, batch_y, batch_ids)

        # Backward pass with gradient scaling
        SCALER.scale(loss).backward()
        SCALER.step(optimizer)
        SCALER.update()

        losses.append(loss.item())

    return np.mean(losses)


def validate(model, X, y):
    """
    Compute validation MSE.
    
    Args:
        model: SimpleMLP instance
        X: [N, 32] standardized features
        y: [N] targets
    
    Returns:
        mse: Mean squared error
    """
    model.eval()
    with torch.no_grad():
        X_t = torch.from_numpy(X).to(DEVICE)
        y_t = torch.from_numpy(y).to(DEVICE)
        pred = model(X_t)
        mse = nn.functional.mse_loss(pred, y_t).item()
    return mse


print("✅ Training functions defined")

✅ Training functions defined


### 🎲 Train Ensemble Models

Training 3 models with different random seeds for ensemble diversity.

In [16]:
# ===========================
# 5.4 TRAIN ENSEMBLE
# ===========================

models = []

print("\n🎲 Training ensemble...\n")
print("=" * 60)

for seed in ENSEMBLE_SEEDS:
    set_seed(seed)
    
    # Initialize model
    model = SimpleMLP(in_dim=X_tr_s.shape[1]).to(DEVICE)
    optimizer = torch.optim.AdamW(model.parameters(), lr=BASE_LR, weight_decay=WEIGHT_DECAY)

    print(f"\n🌱 Seed {seed}:")
    
    for ep in range(1, EPOCHS + 1):
        t0 = time.time()
        
        # Train
        train_loss = train_one_epoch(model, X_tr_s, y_tr, id_tr, optimizer, loss_fn)
        
        # Validate
        val_mse = validate(model, X_va_s, y_va) if X_va_s is not None else 0.0
        
        elapsed = time.time() - t0
        
        print(
            f"  Epoch {ep}/{EPOCHS}  │  "
            f"train_loss={train_loss:.5f}  │  "
            f"val_mse={val_mse:.5f}  │  "
            f"time={elapsed:.1f}s"
        )

    models.append(model)
    print(f"  ✓ Model {len(models)}/3 trained")

print("\n" + "=" * 60)
print(f"✅ Ensemble training complete: {len(models)} models ready")


🎲 Training ensemble...


🌱 Seed 42:
  Epoch 1/5  │  train_loss=0.55990  │  val_mse=0.45236  │  time=6.5s
  Epoch 2/5  │  train_loss=0.37504  │  val_mse=0.36698  │  time=5.9s
  Epoch 3/5  │  train_loss=0.31203  │  val_mse=0.34186  │  time=5.9s
  Epoch 4/5  │  train_loss=0.28046  │  val_mse=0.33321  │  time=5.9s
  Epoch 5/5  │  train_loss=0.26218  │  val_mse=0.32627  │  time=6.0s
  ✓ Model 1/3 trained

🌱 Seed 43:
  Epoch 1/5  │  train_loss=0.60407  │  val_mse=0.46624  │  time=5.9s
  Epoch 2/5  │  train_loss=0.39962  │  val_mse=0.39490  │  time=5.9s
  Epoch 3/5  │  train_loss=0.33539  │  val_mse=0.34737  │  time=5.9s
  Epoch 4/5  │  train_loss=0.29974  │  val_mse=0.31747  │  time=5.8s
  Epoch 5/5  │  train_loss=0.27743  │  val_mse=0.30744  │  time=5.7s
  ✓ Model 2/3 trained

🌱 Seed 44:
  Epoch 1/5  │  train_loss=0.53161  │  val_mse=0.44113  │  time=5.7s
  Epoch 2/5  │  train_loss=0.37283  │  val_mse=0.35969  │  time=5.7s
  Epoch 3/5  │  train_loss=0.31537  │  val_mse=0.33753  │  time=5.7

---

## 🔮 Section 6: Inference & Post-Processing

### Inference Pipeline
1. **Predict** with each ensemble model
2. **Average** predictions across models
3. **Optional normalization** within each file (if NORMALIZE_WITHIN_FILE is set)

### Post-Processing Options
- `'off'`: No normalization
- `'mean'`: Center predictions to mean=0 within each file
- `'meanstd'`: Standardize to mean=0, std=1 within each file

This helps account for systematic biases in different graphs.

In [17]:
# ===========================
# 6.1 PREDICTION FUNCTIONS
# ===========================


def predict_one(model, X):
    """
    Get predictions from one model.
    
    Args:
        model: SimpleMLP instance
        X: [N, 32] standardized features
    
    Returns:
        preds: [N] predictions in numpy
    """
    model.eval()
    with torch.no_grad():
        X_t = torch.from_numpy(X).to(DEVICE)
        pred = model(X_t).cpu().numpy()
    return pred


def normalize_within_ids(pred, ids):
    """
    Apply per-file normalization to predictions.
    
    This helps remove systematic biases across different graphs.
    
    Args:
        pred: [N] predictions
        ids: [N] file identifiers
    
    Returns:
        pred_norm: [N] normalized predictions
    """
    mode = NORMALIZE_WITHIN_FILE
    if mode == "off":
        return pred

    df = pd.DataFrame({"id": ids, "p": pred})
    
    if mode == "mean":
        # Center to mean=0 within each file
        df["p"] = df.groupby("id")["p"].transform(lambda x: x - x.mean())
    elif mode == "meanstd":
        # Standardize to mean=0, std=1 within each file
        df["p"] = df.groupby("id")["p"].transform(
            lambda x: (x - x.mean()) / np.maximum(x.std(), 1e-8)
        )
    
    return df["p"].values


def predict_avg(models, X, ids):
    """
    Ensemble prediction: average across all models.
    
    Args:
        models: List of SimpleMLP instances
        X: [N, 32] standardized features
        ids: [N] file identifiers (for optional normalization)
    
    Returns:
        pred_avg: [N] averaged predictions
    """
    preds = []
    for model in models:
        p = predict_one(model, X)
        preds.append(p)
    
    # Average predictions
    p_avg = np.mean(preds, axis=0)
    
    # Optional per-file normalization
    if ids is not None:
        p_avg = normalize_within_ids(p_avg, ids)
    
    return p_avg


print("\n✅ Inference functions defined")
print(f"   Post-processing: {NORMALIZE_WITHIN_FILE}")


✅ Inference functions defined
   Post-processing: meanstd


In [18]:
# ===========================
# 6.2 GENERATE PREDICTIONS
# ===========================

print("\n🔮 Generating predictions...\n")

# Validation predictions (for metrics)
p_va = predict_avg(models, X_va_s, id_va) if X_va_s is not None else None

# Test predictions (for submission)
p_te = predict_avg(models, X_te_s, id_te) if X_te_s is not None else None

print("✅ Predictions complete:")
if p_va is not None:
    print(f"   Validation: {p_va.shape} = {len(p_va):,} configs")
    print(f"               Across {len(np.unique(id_va))} files")
if p_te is not None:
    print(f"   Test:       {p_te.shape} = {len(p_te):,} configs")
    print(f"               Across {len(np.unique(id_te))} files")


🔮 Generating predictions...

✅ Predictions complete:
   Validation: (1042735,) = 1,042,735 configs
               Across 676 files
   Test:       (1420536,) = 1,420,536 configs
               Across 844 files


---

## 📊 Section 7: Evaluation Metrics

### Metrics Explained

#### **Accuracy @ 1 (Acc@1)**
Percentage of files where we correctly predict the best configuration.
```
For each file:
  true_best = argmin(true_runtimes)
  pred_best = argmin(predicted_runtimes)
  correct if true_best == pred_best
```

#### **Average Regret**
How much slower our predicted config is compared to the true best, on average.
```
regret = (runtime_of_predicted_best - runtime_of_true_best) / runtime_of_true_best
```

Lower regret = better predictions!

### Important Note
Metrics are computed on **ORIGINAL runtime scale**, even though we trained on log(runtime).

In [19]:
# ===========================
# 7.1 METRIC FUNCTIONS
# ===========================


def acc1_and_regret(ids, cfg_idx, y_true, y_pred, use_log_target: bool):
    """
    Compute Accuracy@1 and Average Regret.
    
    Args:
        ids: [N] file identifiers
        cfg_idx: [N] config indices
        y_true: [N] true targets (possibly log-scaled)
        y_pred: [N] predicted targets
        use_log_target: Whether y_true is log-scaled
    
    Returns:
        acc: Accuracy@1 (fraction of correct best configs)
        regret: Average regret (on original runtime scale)
    """
    df = pd.DataFrame({"id": ids, "cfg": cfg_idx, "y": y_true, "p": y_pred})

    # Convert to original runtime scale if needed
    if use_log_target:
        y_orig = np.exp(df["y"].values) - 1e-12
    else:
        y_orig = df["y"].values
    df["y_orig"] = y_orig

    # Find best configs per file
    idx_true_min = df.groupby("id")["y"].idxmin()  # Based on true values
    idx_pred_min = df.groupby("id")["p"].idxmin()  # Based on predictions

    # Accuracy@1: Compare config indices
    true_best_cfg = df.loc[idx_true_min, ["id", "cfg"]].set_index("id")["cfg"]
    pred_best_cfg = df.loc[idx_pred_min, ["id", "cfg"]].set_index("id")["cfg"]
    acc = (true_best_cfg == pred_best_cfg).mean()

    # Regret: Compare original runtime values
    best_true_val = df.loc[idx_true_min, ["id", "y_orig"]].set_index("id")["y_orig"]
    pred_true_val = df.loc[idx_pred_min, ["id", "y_orig"]].set_index("id")["y_orig"]
    regret = (
        (pred_true_val - best_true_val) / np.clip(best_true_val, 1e-12, None)
    ).mean()

    return float(acc), float(regret)


print("\n✅ Metric functions defined")


✅ Metric functions defined


In [20]:
# ===========================
# 7.2 COMPUTE METRICS
# ===========================

if p_va is not None:
    print("\n📊 Validation Metrics:\n" + "=" * 60)
    
    acc, reg = acc1_and_regret(
        id_va, cfg_va, y_va, p_va, use_log_target=USE_LOG_TARGET
    )
    
    print(f"\n  Accuracy @ 1:     {acc:.4f}  ({acc*100:.2f}%)")
    print(f"  Average Regret:   {reg:.6f}")
    print(f"\n  Interpretation:")
    print(f"    • We correctly pick the best config {acc*100:.1f}% of the time")
    print(f"    • When wrong, predicted config is ~{reg*100:.2f}% slower on average")
    print("\n" + "=" * 60)
else:
    print("\n⚠️  No validation data available; skipping metrics.")


📊 Validation Metrics:

  Accuracy @ 1:     0.1065  (10.65%)
  Average Regret:   0.190802

  Interpretation:
    • We correctly pick the best config 10.7% of the time
    • When wrong, predicted config is ~19.08% slower on average



---

## 📤 Section 8: Submission Generation

### Output Format
The competition requires a CSV with:
- **ID**: Unique identifier (e.g., `tile:xla:abc123...`)
- **TopConfigs**: Top 5 config indices separated by semicolons (e.g., `"3;15;7;2;11"`)

### Two-Part Submission
1. **Tile predictions** (this notebook): For `tile:xla:*` IDs
2. **Layout predictions** (separate notebook): For `layout:*` IDs

We generate tile predictions, then append the layout rows from sample_submission.csv unchanged.

In [21]:
# ===========================
# 8.1 GENERATE TILE SUBMISSION
# ===========================

print("\n📤 Creating submission file...\n")

assert p_te is not None and len(p_te) == len(id_te), "Missing test predictions."

# Create DataFrame
df_te = pd.DataFrame({"ID": id_te, "cfg": cfg_te, "p": p_te})

print(f"  Test predictions: {len(df_te):,} configs")
print(f"  Unique files: {df_te['ID'].nunique()}")

# Sort by predicted runtime (ascending = faster configs first)
df_sorted = df_te.sort_values(["ID", "p"], ascending=[True, True])

# Take top 5 configs per file
top5 = df_sorted.groupby("ID", as_index=False).head(5)

print(f"  Top-5 configs per file: {len(top5):,} rows")

# Aggregate into semicolon-separated strings
tile_sub = (
    top5.groupby("ID")["cfg"]
    .apply(lambda s: ";".join(s.astype(int).astype(str).tolist()))
    .reset_index()
)

# Format IDs (add "tile:xla:" prefix if missing)
tile_sub["ID"] = tile_sub["ID"].astype(str).map(lambda x: x if ":" in x else f"tile:xla:{x}")
tile_sub.columns = ["ID", "TopConfigs"]

print(f"  Tile submission rows: {len(tile_sub)}")
print(f"\n  Example entries:")
display(tile_sub.head(3))

# Save tile-only submission
tile_path = WORK_DIR / "submission_tile_top5.csv"
tile_sub.to_csv(tile_path, index=False)
print(f"\n  ✅ Saved: {tile_path}")


📤 Creating submission file...

  Test predictions: 1,420,536 configs
  Unique files: 844
  Top-5 configs per file: 4,192 rows
  Tile submission rows: 844

  Example entries:


,ID,TopConfigs
0,tile:xla:0023795810403f8b0b244d88c901322f,464;1511;1638;1850;2367
1,tile:xla:005c91ca7a50fffc663678fd44316f04,117;79;229;252;188
2,tile:xla:0070642211d5a98a16b94f4d7df229fe,254;577;189;805;813



  ✅ Saved: /kaggle/working/submission_tile_top5.csv


In [22]:
# ===========================
# 8.2 MERGE WITH LAYOUT ROWS
# ===========================

print("\n📋 Adding layout predictions from sample submission...\n")

assert SAMPLE_SUB is not None and Path(SAMPLE_SUB).exists(), "sample_submission.csv not found."

# Load sample submission
sample_raw = pd.read_csv(SAMPLE_SUB)
sample_cols = list(sample_raw.columns)

# Standardize column names
sample_df = sample_raw.rename(
    columns={sample_cols[0]: "ID", sample_cols[1]: "TopConfigs"}
)[["ID", "TopConfigs"]]

# Extract layout rows (these are handled by a separate notebook)
layout_rows = sample_df[sample_df["ID"].str.startswith("layout:", na=False)].copy()

print(f"  Layout rows from sample: {len(layout_rows)}")

# Combine tile + layout
final = pd.concat([tile_sub, layout_rows], ignore_index=True)

# Restore original column names
final = final.rename(columns={"ID": sample_cols[0], "TopConfigs": sample_cols[1]})

print(f"  Final submission rows: {len(final)}")
print(f"    • Tile:   {len(tile_sub)}")
print(f"    • Layout: {len(layout_rows)}")

# Save final submission
final_path = WORK_DIR / "submission.csv"
final.to_csv(final_path, index=False)

print(f"\n  ✅ Saved: {final_path}")
print("\n  Preview:")
display(final.head(5))


📋 Adding layout predictions from sample submission...

  Layout rows from sample: 50
  Final submission rows: 894
    • Tile:   844
    • Layout: 50

  ✅ Saved: /kaggle/working/submission.csv

  Preview:


,ID,TopConfigs
0,tile:xla:0023795810403f8b0b244d88c901322f,464;1511;1638;1850;2367
1,tile:xla:005c91ca7a50fffc663678fd44316f04,117;79;229;252;188
2,tile:xla:0070642211d5a98a16b94f4d7df229fe,254;577;189;805;813
3,tile:xla:008191e0c67a6e7a62cde1a3e1d66795,1115;1051;349;635;444
4,tile:xla:008730b43f100be7c2800d7cb89578a4,451;736;281;316;987
